# Wagner Whiten Model
## Catherine Christopher

In [5]:
import pandas as pd
import numpy as np

In [6]:
K = 80
h = 1.75

demandlist = [36, 60, 85, 11, 39, 75]

tempDict = {'Period':np.arange(1,len(demandlist)+1),
            'Demand':demandlist}    
demanddf = pd.DataFrame(tempDict)
demanddf = demanddf.set_index('Period')

In [7]:
def findcost(t,j,K,h,demanddf,repeatdf):
    cost = 0
    tj = t + j
    
    for k in np.arange(1,j+1):
        t1k = t - 1 + k
        periodcost = ((2*k - 1)*demanddf.loc[t1k,'Demand'])/2
        cost += periodcost
    v_t = K + h*cost
    if tj != len(demanddf)+1:
        v_t += repeatlist[len(demanddf) - tj][2]
    return v_t

In [8]:
repeatlist = []
for t in demanddf.index[::-1]:
    steps = np.arange(1,len(demanddf.index) + 2 - t )
    tempDict = {}
    for j in steps:
        v_t = findcost(t, j, K, h, demanddf, repeatlist)
        tempDict[v_t] = j
    V_t = min(list(tempDict.keys()))
    repeatlist.append([t, tempDict[V_t], V_t, False])
repeatdf = pd.DataFrame(repeatlist, columns = ['t','j*_t','V_t','truefalse'])
repeatdf = repeatdf.set_index('t').sort_index()   
totalcost = repeatdf.loc[1,'V_t']

solutionlist = []
for t in demanddf.index:
    if repeatdf.loc[t,'truefalse']:
        solutionlist.append([t, 0, 0])
    else:
        j = repeatdf.loc[t,'j*_t']
        orderSize = demanddf.loc[t: t - 1 + j,'Demand'].sum()
        solutionlist.append([t, j, orderSize])
        for i in range(int(t - 1),int(t + j)):
            repeatdf.loc[i,'truefalse'] = True
        
solutiondf = pd.DataFrame(solutionlist, columns = ['Period', 'Periods Covered', 'Order Quantity'])
solutiondf = solutiondf.set_index('Period')

print(solutiondf)

        Periods Covered  Order Quantity
Period                                 
1                   1.0              36
2                   1.0              60
3                   2.0              96
4                   0.0               0
5                   1.0              39
6                   1.0              75
